### MERGE ZILLOW TO REALTOR

In [2]:
import pandas as pd
import os


In [47]:
# load the zillow long data
zillow_long = pd.read_csv('../Data/zillow/merged_zillow_long.csv')
# load the realtor data
realtor_data = pd.read_csv('../Data/realtor/realtor_zip_monthly_historical.csv')

# print head of zillow_long
print(zillow_long.head())

# print head of realtor_data
print(realtor_data.head())

/var/folders/sp/_mjrzq3j7cx5cnmtdrl_njsr0000gn/T/ipykernel_71215/623675677.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  realtor_data = pd.read_csv('../Data/realtor/realtor_zip_monthly_historical.csv')


   Zip_Code           Metro_Name       Month           ZHVI  mean_doz  invt  \
0      1001  Springfield, MA, MA  2000-01-31  122934.022744       NaN   NaN   
1      1001  Springfield, MA, MA  2000-02-29  122887.360088       NaN   NaN   
2      1001  Springfield, MA, MA  2000-03-31  123035.564987       NaN   NaN   
3      1001  Springfield, MA, MA  2000-04-30  123370.688429       NaN   NaN   
4      1001  Springfield, MA, MA  2000-05-31  124237.694503       NaN   NaN   

   sales  market_temp  new  
0    NaN          NaN  NaN  
1    NaN          NaN  NaN  
2    NaN          NaN  NaN  
3    NaN          NaN  NaN  
4    NaN          NaN  NaN  
  month_date_yyyymm  postal_code            zip_name  median_listing_price  \
0            202408      32343.0          midway, fl              304950.0   
1            202408      62896.0  west frankfort, il              125000.0   
2            202408      56477.0          sebeka, mn              302400.0   
3            202408      25315.0      c

In [50]:
# for the zillow data, we want to convert the end of month date to the first of the next month

# lets start by converting the zillow_long Month to a datetime object of format yyyy-mm-dd
zillow_long['Month'] = pd.to_datetime(zillow_long['Month'], format='%Y-%m-%d')

# lets make the end of month date the first day of the next month
zillow_long['Month'] = zillow_long['Month'] + pd.DateOffset(months=1)
# now make it the first day of that month
zillow_long['Month'] = pd.to_datetime(zillow_long['Month']).dt.to_period('M').dt.to_timestamp()

# print first 10 months
print(zillow_long['Month'].head(10))

# remove realtor rows with the month that is not in month format
realtor_data['month_date_yyyymm'] = pd.to_datetime(realtor_data['month_date_yyyymm'], format='%Y%m', errors='coerce')

# remove rows with NaN in month_date_yyyymm
realtor_data = realtor_data.dropna(subset=['month_date_yyyymm'])

# print months of the realtor data
print(realtor_data['month_date_yyyymm'].head(10))









0   2000-03-01
1   2000-04-01
2   2000-05-01
3   2000-06-01
4   2000-07-01
5   2000-08-01
6   2000-09-01
7   2000-10-01
8   2000-11-01
9   2000-12-01
Name: Month, dtype: datetime64[ns]
0   2024-08-01
1   2024-08-01
2   2024-08-01
3   2024-08-01
4   2024-08-01
5   2024-08-01
6   2024-08-01
7   2024-08-01
8   2024-08-01
9   2024-08-01
Name: month_date_yyyymm, dtype: datetime64[ns]


In [51]:
# merge the zillow_long and realtor_data on Month and Zip

# rename the month_date_yyyymm column to Month, and the postal_code column to Zip_Code
realtor_data.rename(columns={'month_date_yyyymm': 'Month', 'postal_code': 'Zip_Code'}, inplace=True)

# head of realtor_data
print(realtor_data.head())

# print the Zip_Code column for both dataframes
print(zillow_long['Zip_Code'].unique())
print(realtor_data['Zip_Code'].unique())

merged_data = pd.merge(zillow_long, realtor_data, on=['Month', 'Zip_Code'], how='inner')

# print head of merged_data
print(merged_data.head())

# print the shape of merged_data
print(merged_data.shape)



       Month  Zip_Code            zip_name  median_listing_price  \
0 2024-08-01   32343.0          midway, fl              304950.0   
1 2024-08-01   62896.0  west frankfort, il              125000.0   
2 2024-08-01   56477.0          sebeka, mn              302400.0   
3 2024-08-01   25315.0      charleston, wv               65000.0   
4 2024-08-01   76454.0          gorman, tx              184500.0   

   median_listing_price_mm  median_listing_price_yy  active_listing_count  \
0                   0.0168                  -0.1028                  14.0   
1                   0.3999                   0.6955                  29.0   
2                  -0.1186                  -0.1919                  13.0   
3                  -0.1613                   0.1017                   5.0   
4                  -0.0027                   0.1535                   3.0   

   active_listing_count_mm  active_listing_count_yy  median_days_on_market  \
0                  -0.2632                  -0.125